In [ ]:
import os
import shutil
import numpy as np

In [ ]:
binary_classifier_flag = True
# binary_classifier_flag = False
segment_interval = 4
if binary_classifier_flag:
    old_save_root = os.path.join('/datadrive', 'TUSZ', 'new_3_TUSZ_processed_binary_individual_segments')
    new_save_root = os.path.join('/datadrive', 'TUSZ', 'new_3_balanced_TUSZ_processed_binary_individual_segments')
else:
    old_save_root = os.path.join('/datadrive', 'TUSZ', 'new_3_TUSZ_processed_multiclass_individual_segments')
    new_save_root = os.path.join('/datadrive', 'TUSZ', 'new_3_balanced_TUSZ_processed_multiclass_individual_segments')

if binary_classifier_flag:
    seizure_types = ['bckg', 'seizure']
    seizure_file_count = [0, 0]
    seizure_sample_count = [0, 0]
    patient_count = [0, 0]
    # seizure_session_downsampling_ratio = [20, 1]
else:
    seizure_types = ['fnsz', 'gnsz', 'cpsz', 'bckg']
    seizure_file_count = [0, 0, 0, 0]
    seizure_sample_count = [0, 0, 0, 0]
    patient_count = [0, 0, 0, 0]
    # seizure_session_downsampling_ratio = [1, 1, 1, 10]

for i, seizure_type in enumerate(seizure_types):
    old_folder = os.path.join(old_save_root, 'segment_interval_'+str(segment_interval)+'_sec', 'train', seizure_type)
    old_files = os.listdir(old_folder)
    seizure_file_count[i] = len(old_files)
    for old_file in old_files:
        this_EEG_data = np.load(os.path.join(old_folder, old_file))
        seizure_sample_count[i] += this_EEG_data.shape[0]
    unique_patients = set()
    for file_name in old_files:
        this_patient = file_name.split('_s')[0]
        if this_patient not in unique_patients:
            unique_patients.add(this_patient)
    patient_count[i] = len(unique_patients)

print(seizure_sample_count)
print(seizure_file_count)
print(patient_count)

In [ ]:
if binary_classifier_flag:
    print(seizure_sample_count[0]/seizure_sample_count[1])
else:
    for i in range(len(seizure_types)-1):
        print(seizure_sample_count[-1]/seizure_sample_count[i])

In [ ]:
seizure_session_downsampling_ratio = []
for i, seizure_type in enumerate(seizure_types):
    seizure_session_downsampling_ratio.append(min(seizure_sample_count)/seizure_sample_count[i])

seizure_session_downsampling_ratio

In [ ]:
import random
# random.seed(2023)


for i, seizure_type in enumerate(seizure_types):
    for train_val_test_mode in ['train', 'val', 'test']:
        old_folder = os.path.join(old_save_root, 'segment_interval_'+str(segment_interval)+'_sec', train_val_test_mode, seizure_type)
        old_files = os.listdir(old_folder)
        new_folder = os.path.join(new_save_root, 'segment_interval_'+str(segment_interval)+'_sec', train_val_test_mode, seizure_type)
        if not os.path.exists(new_save_root):
            os.mkdir(new_save_root)
        if not os.path.exists(os.path.join(new_save_root, 'segment_interval_'+str(segment_interval)+'_sec')):
            os.mkdir(os.path.join(new_save_root, 'segment_interval_'+str(segment_interval)+'_sec'))
        if not os.path.exists(os.path.join(new_save_root, 'segment_interval_'+str(segment_interval)+'_sec', train_val_test_mode)):
            os.mkdir(os.path.join(new_save_root, 'segment_interval_'+str(segment_interval)+'_sec', train_val_test_mode))
        if not os.path.exists(new_folder):
            os.mkdir(new_folder)
        for old_file in old_files:
            if train_val_test_mode in ['train', 'val']:
                # if random.randint(1, seizure_session_downsampling_ratio[i]) <= 1:                
                    # shutil.copy(os.path.join(old_folder, old_file), os.path.join(new_folder, old_file))
                this_EEG_data = np.load(os.path.join(old_folder, old_file))
                new_EEG_data = []
                new_EEG_indices = []
                for this_index in range(this_EEG_data.shape[0]):
                    if random.random() <= seizure_session_downsampling_ratio[i]:
                        new_EEG_indices.append(this_index)
                # print('Selected probability: ', seizure_session_downsampling_ratio[i])
                # print('Original size: ', this_EEG_data.shape)                

                new_EEG_data = this_EEG_data[new_EEG_indices, :, :]
                # print('Selected size: ', new_EEG_data.shape)

                np.save(os.path.join(new_folder, old_file), new_EEG_data)
            else:                
                shutil.copy(os.path.join(old_folder, old_file), os.path.join(new_folder, old_file))
                pass

In [ ]:
if binary_classifier_flag:
    new_seizure_file_count = [0, 0]
    new_seizure_sample_count = [0, 0]
    new_patient_count = [0, 0]
else:
    new_seizure_file_count = [0, 0, 0, 0]
    new_seizure_sample_count = [0, 0, 0, 0]
    new_patient_count = [0, 0, 0, 0]

for i, seizure_type in enumerate(seizure_types):
    new_folder = os.path.join(new_save_root, 'segment_interval_'+str(segment_interval)+'_sec', 'train', seizure_type)
    new_files = os.listdir(new_folder)
    new_seizure_file_count[i] = len(new_files)
    for new_file in new_files:
        this_EEG_data = np.load(os.path.join(new_folder, new_file))
        new_seizure_sample_count[i] += this_EEG_data.shape[0]
    unique_patients = set()
    for file_name in new_files:
        this_patient = file_name.split('_s')[0]
        if this_patient not in unique_patients:
            unique_patients.add(this_patient)
    new_patient_count[i] = len(unique_patients)

print(new_seizure_sample_count)
print(new_seizure_file_count)
print(new_patient_count)